# Calculating the July 2025 Earthquake data from PHIVOLCS

## This section reads the file and computes for the mean/min/max and number of events

In [1]:
import pandas as pd #import the panda first
fp = 'July_2025_PHIVOLCS.csv' # make sure this file is uploaded in your respository to run. Also note that this file must be UTF-8 coded

# YOUR CODE HERE
data = pd.read_csv(fp)

In [2]:
data.head() #Check the headings

,Date,Time,Latitude,Longitude,depth,Mag,Location
0,31-Jul-25,11:50 PM,8.06,126.25,33,1.6,018 km N 84° W of Lingig (Surigao Del Sur)
1,31-Jul-25,11:50 PM,8.63,126.36,22,1.4,020 km S 49° E of San Agustin (Surigao Del Sur)
2,31-Jul-25,11:46 PM,9.78,123.02,26,2.0,011 km N 65° E of Mabinay (Negros Oriental)
3,31-Jul-25,11:45 PM,8.32,126.55,45,1.6,024 km S 78° E of Hinatuan (Surigao Del Sur)
4,31-Jul-25,11:27 PM,13.40,120.17,7,2.7,032 km S 86° W of Paluan (Occidental Mindoro)


In [3]:
row_count = None
column_names = None
column_datatypes = None

# YOUR CODE HERE
row_count = len(data) #use to identify number of rows
column_names = data.columns #use to determine column names
column_datatypes = data.dtypes #use to determine types of columns

In [4]:
# Print the number of rows in the dataframe:
print(f"There are {row_count} rows")

# Print the column names:
print(f"The columns are: \n{column_names}")

# Print the column datatypes:
print(f"The column types are: \n{column_datatypes}")

There are 1421 rows
The columns are: 
Index(['Date', 'Time', 'Latitude', 'Longitude', 'depth', 'Mag', 'Location'], dtype='object')
The column types are: 
Date          object
Time          object
Latitude     float64
Longitude    float64
depth          int64
Mag          float64
Location      object
dtype: object


### For the Magnitude data, this computes mean/min/max and also unique days

In [5]:
## Change the column names
data.columns = data.columns.str.replace("Mag", "Mw")

In [6]:
print(data.columns)

Index(['Date', 'Time', 'Latitude', 'Longitude', 'depth', 'Mw', 'Location'], dtype='object')


In [7]:
data.head() #check if it has been replaced

,Date,Time,Latitude,Longitude,depth,Mw,Location
0,31-Jul-25,11:50 PM,8.06,126.25,33,1.6,018 km N 84° W of Lingig (Surigao Del Sur)
1,31-Jul-25,11:50 PM,8.63,126.36,22,1.4,020 km S 49° E of San Agustin (Surigao Del Sur)
2,31-Jul-25,11:46 PM,9.78,123.02,26,2.0,011 km N 65° E of Mabinay (Negros Oriental)
3,31-Jul-25,11:45 PM,8.32,126.55,45,1.6,024 km S 78° E of Hinatuan (Surigao Del Sur)
4,31-Jul-25,11:27 PM,13.40,120.17,7,2.7,032 km S 86° W of Paluan (Occidental Mindoro)


In [8]:
# in removing a column, there are other ways and one of them is just by using drop()
data = data.drop(['Location','Longitude','Latitude','Time'], axis=1)

In [9]:
data.head() #check if it has been updated

,Date,depth,Mw
0,31-Jul-25,33,1.6
1,31-Jul-25,22,1.4
2,31-Jul-25,26,2.0
3,31-Jul-25,45,1.6
4,31-Jul-25,7,2.7


In [10]:
# convert the date
data["Date"] = pd.to_datetime(data["Date"])

C:\Users\User\AppData\Local\Temp\ipykernel_5536\3668008331.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"])


In [11]:
data["Date"].head()

0   2025-07-31
1   2025-07-31
2   2025-07-31
3   2025-07-31
4   2025-07-31
Name: Date, dtype: datetime64[ns]

In [12]:
data.head()

,Date,depth,Mw
0,2025-07-31,33,1.6
1,2025-07-31,22,1.4
2,2025-07-31,26,2.0
3,2025-07-31,45,1.6
4,2025-07-31,7,2.7


In [13]:
data["MONTH_DAY"] = pd.to_datetime(data["Date"], format="%d %B", exact=False).dt.strftime('%m%d')

In [14]:
data.head() # check if you added a new column with converted date

,Date,depth,Mw,MONTH_DAY
0,2025-07-31,33,1.6,0731
1,2025-07-31,22,1.4,0731
2,2025-07-31,26,2.0,0731
3,2025-07-31,45,1.6,0731
4,2025-07-31,7,2.7,0731


In [15]:
# group the data per day
grouped = data.groupby(data["MONTH_DAY"])

In [16]:
type(grouped)
len(grouped)

31

In [17]:
grouped.groups.keys()

dict_keys(['0701', '0702', '0703', '0704', '0705', '0706', '0707', '0708', '0709', '0710', '0711', '0712', '0713', '0714', '0715', '0716', '0717', '0718', '0719', '0720', '0721', '0722', '0723', '0724', '0725', '0726', '0727', '0728', '0729', '0730', '0731'])

In [18]:
# Create an empty DataFrame for the aggregated values
daily_max_list = []

# Columns we want to aggregate
max_cols = ["Mw", "depth"]

# Iterate over the groups
for key, group in grouped:
    # Calculate max values
    max_values = group[max_cols].max()

    # Add the key (date/month info)
    max_values["MONTH_DAY"] = key

    # Save to list
    daily_max_list.append(max_values)

# Turn into DataFrame
daily_max = pd.DataFrame(daily_max_list).reset_index(drop=True)

print(daily_max.head())

    Mw  depth MONTH_DAY
0  3.8  106.0      0701
1  4.0  178.0      0702
2  3.8  148.0      0703
3  4.7  168.0      0704
4  3.1  108.0      0705


In [19]:
grouped = data.groupby(data["MONTH_DAY"])

daily_stats = round(grouped[['Mw','depth']].mean(), 2)
daily_stats = daily_stats.rename(columns={'Mw': 'Mw_mean','depth': 'DEP_mean'})

daily_stats[['Mw_min', 'DEP_min']] = grouped[['Mw','depth']].min()
daily_stats[['Mw_max', 'DEP_max']] = grouped[['Mw','depth']].max()

daily_stats['N_events'] = grouped['Mw'].count()
mean_Nevents = daily_stats['N_events'].mean()

In [20]:
daily_stats.head()

,Mw_mean,DEP_mean,Mw_min,DEP_min,Mw_max,DEP_max,N_events
MONTH_DAY,,,,,,,
0701,2.35,27.06,1.5,2,3.8,106,50
0702,2.21,41.57,1.3,2,4.0,178,35
0703,2.33,34.49,1.4,2,3.8,148,39
0704,2.39,34.15,1.4,2,4.7,168,47
0705,2.20,27.88,1.0,1,3.1,108,42


In [21]:
 # Print info about the current input file:
print(f"MONTH: July 2025")
print(f"NUMBER OF EVENTS RECORDED: {len(data)}")
    
print(f'Average number of earthquakes per day: {round(mean_Nevents, 1)}') # round to the nearest 1 decimal place
print(daily_stats.head())

    # Print info
print(daily_stats.sort_values(by="N_events", ascending=False).head(5))
print("\n")

MONTH: July 2025
NUMBER OF EVENTS RECORDED: 1421
Average number of earthquakes per day: 45.8
           Mw_mean  DEP_mean  Mw_min  DEP_min  Mw_max  DEP_max  N_events
MONTH_DAY                                                               
0701          2.35     27.06     1.5        2     3.8      106        50
0702          2.21     41.57     1.3        2     4.0      178        35
0703          2.33     34.49     1.4        2     3.8      148        39
0704          2.39     34.15     1.4        2     4.7      168        47
0705          2.20     27.88     1.0        1     3.1      108        42
           Mw_mean  DEP_mean  Mw_min  DEP_min  Mw_max  DEP_max  N_events
MONTH_DAY                                                               
0707          2.28     23.89     1.2        1     4.0      132        73
0714          2.29     46.00     1.3        2     5.1      529        69
0722          2.50     22.28     1.4        1     4.0       78        61
0731          2.32     34.07   